In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
df_origin = pd.read_csv('data/train.csv')
df_origin_test = pd.read_csv('data/test.csv')

# Import Encoded Dataset

In [3]:
df = pd.read_csv('Final Train.csv')
df_comp = pd.read_csv('Final Test.csv')

# Split Dataset

In [4]:
X = df.drop(columns=['row_id', 'date', 'num_sold'])
y = df[['num_sold']]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25)

# Create Models

### Random Forest

In [6]:
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

In [18]:
rf = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = .1,
                max_depth = 7, alpha = 16, n_estimators = 50)
#rf = RandomForestRegressor(max_depth = 20, min_samples_leaf = 4, n_estimators=1000)

In [19]:
rf.fit(X_train, y_train)

XGBRegressor(alpha=16, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=7, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=50, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=16, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [20]:
prediction = rf.predict(X_test)

In [21]:
rf.score(X_test, y_test)

0.9493928462641795

# Tune Dataset

from sklearn.model_selection import GridSearchCV

param_grid = {'n_estimators': [100, 500, 1000],
              'min_samples_leaf': [1, 2, 4],
              'max_depth': [5, 10, 20],
              'bootstrap': [True, False]
}

grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(X_train, y_train)

grid_search.best_estimator_

# Comp Prediction

In [11]:
comp_pred = rf.predict(df_comp.drop(columns=['date', 'row_id']))

In [12]:
comp_pred = np.around(comp_pred)

In [13]:
df_final = df_comp

In [14]:
df_final['num_sold'] = comp_pred
df_final['num_sold'] = df_final['num_sold'].astype(int)

In [15]:
#sns.set(rc = {'figure.figsize':(15,8)})
#sns.lineplot(df['date'], df['num_sold'], hue=df_origin['product'])
#sns.lineplot(df_comp['date'], df_final['num_sold'], hue=df_origin_test['product'])

In [16]:
df_final = df_final[['row_id', 'num_sold']]

In [17]:
df_final.to_csv('Algo Submission.csv', index=False)

1st - .797, 2nd - .897, 3rd - .907